In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras import models
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Dense, ConvLSTM2D, MultiHeadAttention, Flatten, MaxPooling3D

import os
import random
from pathlib import Path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DIR = "drive/MyDrive/dataset"

In [ ]:
# cek
os.listdir(DIR)

['train', 'val']

In [ ]:
print("file train/fight: ",len(os.listdir(DIR+"/train/Fight")))
print("file train/nonfight:", len(os.listdir(DIR+"/train/NonFight")))
print("file val/fight: ", len(os.listdir(DIR+"/val/Fight")))
print("file val/nonfight: ", len(os.listdir(DIR+"/val/NonFight")))

file train/fight:  800
file train/nonfight: 800
file val/fight:  200
file val/nonfight:  200


In [ ]:
class FrameGenerator:
    def __init__(self, path, training=False):
        """ Returns a set of frames with their associated label.
          Args:
            path: Video file paths.
            n_frames: Number of frames.
            training: Boolean to determine if training dataset is being created.
        """
        self.path = path

        self.training = training
        self.class_names = ['NonFight', 'Fight']
        self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

    def get_files_and_class_names(self):
        if self.training:
            video_paths = list(self.path.glob('Fight/*.npy'))[:160] + list(self.path.glob('NonFight/*.npy'))[:160]
        else:
            video_paths = list(self.path.glob('Fight/*.npy'))[:40] + list(self.path.glob('NonFight/*.npy'))[:40]
        classes = [p.parent.name for p in video_paths]

        return video_paths, classes

    def __call__(self):
        video_paths, classes = self.get_files_and_class_names()

        pairs = list(zip(video_paths, classes))

        if self.training:
            random.shuffle(pairs)

        for path, name in pairs:
            video_frames = self.read_npy_file(path)
            label = self.class_ids_for_name[name]
            yield video_frames, label


    def read_npy_file(self, path):
        data = np.load(path)
        data = np.float32(data)
        data = data/255.0
        return data

In [ ]:
train_file = Path(DIR).joinpath("train").with_suffix('')
test_file = Path(DIR).joinpath("val").with_suffix('')
train_file

PosixPath('drive/MyDrive/dataset/train')

In [ ]:
fg = FrameGenerator(train_file, training=True)

# test
frames, label = next(fg())
print(fg)
print(f"Shape: {frames.shape}")
print(f"Label: {label}")

Shape: (21, 224, 224, 3)
Label: 0


In [ ]:
# buat tf.data.Dataset
batch_size = 3

output_signature = (tf.TensorSpec(shape=(None, None, None, 3), dtype=tf.float32),
                    tf.TensorSpec(shape=(), dtype=tf.int16))

train_ds = tf.data.Dataset.from_generator(FrameGenerator(train_file, training=True),
                                          output_signature=output_signature)


test_ds = tf.data.Dataset.from_generator(FrameGenerator(test_file),
                                          output_signature=output_signature)


In [ ]:
train_ds = train_ds.batch(batch_size)
test_ds = test_ds.batch(batch_size)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size = AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size = AUTOTUNE)

In [ ]:
# test data
for frames, labels in train_ds.take(10):
    print(labels)

print(f"Shape: {frames.shape}")
print(f"Label: {labels.shape}")

tf.Tensor([0 0 1], shape=(3,), dtype=int16)
tf.Tensor([1 0 0], shape=(3,), dtype=int16)
tf.Tensor([0 1 1], shape=(3,), dtype=int16)
tf.Tensor([1 0 1], shape=(3,), dtype=int16)
tf.Tensor([1 0 1], shape=(3,), dtype=int16)
tf.Tensor([0 0 0], shape=(3,), dtype=int16)
tf.Tensor([1 0 1], shape=(3,), dtype=int16)
tf.Tensor([1 1 1], shape=(3,), dtype=int16)
tf.Tensor([1 0 0], shape=(3,), dtype=int16)
tf.Tensor([1 1 0], shape=(3,), dtype=int16)
Shape: (3, 21, 224, 224, 3)
Label: (3,)


In [ ]:
# RESNET pretrained
pretrained_model = tf.keras.applications.ResNet50(include_top=False,
                                                  input_shape=(224, 224, 3),
                                                  pooling='avg',
                                                  weights='imagenet')

In [ ]:
# buang avg pool layer
gap_layer_name = 'avg_pool'
gap_layer_index = None

for i, layer in enumerate(pretrained_model.layers):
    if layer.name == gap_layer_name:
        gap_layer_index = i
        break


if gap_layer_index is not None:
    model_without_gap = tf.keras.Model(inputs=pretrained_model.input, outputs=pretrained_model.layers[gap_layer_index - 1].output)

else:
    print("GAP layer not found.")

In [ ]:
video_input_shape = (21, 224, 224, 3)

video_input = tf.keras.Input(shape=video_input_shape)
time_distributed = TimeDistributed(model_without_gap)(video_input)

conv_lstm_output = ConvLSTM2D(filters=256, kernel_size=(3, 3), padding="same", return_sequences=True)(time_distributed)

mhsa_output = MultiHeadAttention(num_heads=8, key_dim=256, value_dim=256)(conv_lstm_output, conv_lstm_output)
attention_time = TimeDistributed(tf.keras.layers.Lambda(lambda x: x))(mhsa_output)

conv_lstm_output2 = ConvLSTM2D(filters=256, kernel_size=(3, 3), padding="same", return_sequences=True)(attention_time)

maxpool3d = MaxPooling3D(pool_size=(1,2,2))(conv_lstm_output2)

flattened = Flatten()(maxpool3d)

dense_1 = Dense(1000, activation="relu")(flattened)
dense_2 = Dense(256, activation="relu")(dense_1)
dense_3 = Dense(10, activation="relu")(dense_2)
dense_4 = Dense(2, activation="softmax")(dense_3)

# Create the model
model = tf.keras.Model(inputs=video_input, outputs=dense_4)

model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 21, 224, 224, 3)]    0         []                            
                                                                                                  
 time_distributed_4 (TimeDi  (None, 21, 7, 7, 2048)       2358771   ['input_6[0][0]']             
 stributed)                                               2                                       
                                                                                                  
 conv_lstm2d_4 (ConvLSTM2D)  (None, 21, 7, 7, 256)        2123468   ['time_distributed_4[0][0]']  
                                                          8                                       
                                                                                            

In [ ]:
epochs = 20
lr = 1e-4

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr),
              metrics=['accuracy'])

In [ ]:
checkpoint_filepath = 'model_checkpoint.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,  # Set to True if you only want to save weights
    save_freq='epoch',
    period=5  # Save every 5 epochs
)

In [ ]:
results = model.fit(train_ds,
                    validation_data=test_ds,
                    epochs=epochs,
                    validation_freq=1,
                    verbose=1, callbacks=[model_checkpoint_callback])

Epoch 1/20
107/107 [==============================] - 294s 2s/step - loss: 0.6987 - accuracy: 0.4719 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/20
107/107 [==============================] - 167s 2s/step - loss: 0.6932 - accuracy: 0.4844 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/20
107/107 [==============================] - 167s 2s/step - loss: 0.6932 - accuracy: 0.4781 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/20
107/107 [==============================] - 167s 2s/step - loss: 0.6932 - accuracy: 0.4812 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/20
107/107 [==============================] - 183s 2s/step - loss: 0.6932 - accuracy: 0.4906 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 6/20
107/107 [==============================] - 168s 2s/step - loss: 0.6932 - accuracy: 0.4906 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 7/20
107/107 [==============================] - 167s 2s/step - loss: 0.6932 - accuracy: 0.4969 - val_loss: 0.6931 - val_accuracy: 0.5000

In [ ]:
model.evaluate(test_ds, return_dict=True)

27/27 [==============================] - 11s 390ms/step - loss: 0.6931 - accuracy: 0.5000


{'loss': 0.6931478977203369, 'accuracy': 0.5}

In [ ]:
# RESNET pretrained
pretrained_model = tf.keras.applications.ResNet50(include_top=False,
                                                  input_shape=(224, 224, 3),
                                                  pooling='avg',
                                                  weights='imagenet')

In [ ]:
# buang avg pool layer
gap_layer_name = 'avg_pool'
gap_layer_index = None

for i, layer in enumerate(pretrained_model.layers):
    if layer.name == gap_layer_name:
        gap_layer_index = i
        break


if gap_layer_index is not None:
    model_without_gap = tf.keras.Model(inputs=pretrained_model.input, outputs=pretrained_model.layers[gap_layer_index - 1].output)

else:
    print("GAP layer not found.")

In [ ]:
video_input_shape = (21, 224, 224, 3)

video_input = tf.keras.Input(shape=video_input_shape)
time_distributed = TimeDistributed(model_without_gap)(video_input)

conv_lstm_output = ConvLSTM2D(filters=256, kernel_size=(3, 3), padding="same", return_sequences=True)(time_distributed)

# mhsa_output = MultiHeadAttention(num_heads=8, key_dim=256, value_dim=256)(conv_lstm_output, conv_lstm_output)
# attention_time = TimeDistributed(tf.keras.layers.Lambda(lambda x: x))(mhsa_output)

# conv_lstm_output2 = ConvLSTM2D(filters=256, kernel_size=(3, 3), padding="same", return_sequences=True)(attention_time)

maxpool3d = MaxPooling3D(pool_size=(1,2,2))(conv_lstm_output)

flattened = Flatten()(maxpool3d)

dense_1 = Dense(1000, activation="relu")(flattened)
dense_2 = Dense(256, activation="relu")(dense_1)
dense_3 = Dense(10, activation="relu")(dense_2)
dense_4 = Dense(2, activation="softmax")(dense_3)

# Create the model
model = tf.keras.Model(inputs=video_input, outputs=dense_4)

model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 21, 224, 224, 3   0         
                             )]                                  
                                                                 
 time_distributed_7 (TimeDi  (None, 21, 7, 7, 2048)    23587712  
 stributed)                                                      
                                                                 
 conv_lstm2d_7 (ConvLSTM2D)  (None, 21, 7, 7, 256)     21234688  
                                                                 
 max_pooling3d_4 (MaxPoolin  (None, 21, 3, 3, 256)     0         
 g3D)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 48384)             0         
                                                           

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
              metrics=['accuracy'])

In [ ]:
checkpoint_filepath = 'model_checkpoint2.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,  # Set to True if you only want to save weights
    save_freq='epoch',
    period=5  # Save every 5 epochs
)

In [ ]:
results = model.fit(train_ds,
                    validation_data=test_ds,
                    epochs=epochs,
                    validation_freq=1,
                    verbose=1, callbacks=[model_checkpoint_callback])

Epoch 1/20
107/107 [==============================] - 198s 1s/step - loss: 0.7559 - accuracy: 0.6125 - val_loss: 0.7391 - val_accuracy: 0.5000
Epoch 2/20
107/107 [==============================] - 159s 1s/step - loss: 0.4839 - accuracy: 0.7656 - val_loss: 1.3274 - val_accuracy: 0.5000
Epoch 3/20
107/107 [==============================] - 159s 1s/step - loss: 0.3584 - accuracy: 0.8562 - val_loss: 0.9411 - val_accuracy: 0.5250
Epoch 4/20
107/107 [==============================] - 159s 1s/step - loss: 0.3072 - accuracy: 0.8875 - val_loss: 1.1976 - val_accuracy: 0.5500
Epoch 5/20
107/107 [==============================] - 168s 2s/step - loss: 0.2439 - accuracy: 0.9156 - val_loss: 0.9767 - val_accuracy: 0.7375
Epoch 6/20
107/107 [==============================] - 159s 1s/step - loss: 0.1262 - accuracy: 0.9531 - val_loss: 1.5305 - val_accuracy: 0.6750
Epoch 7/20
107/107 [==============================] - 159s 1s/step - loss: 0.1300 - accuracy: 0.9469 - val_loss: 0.8788 - val_accuracy: 0.7375

In [ ]:
model.evaluate(test_ds, return_dict=True)

27/27 [==============================] - 9s 340ms/step - loss: 1.8265 - accuracy: 0.6625


{'loss': 1.8264859914779663, 'accuracy': 0.6625000238418579}